# 1. Data collection

## 1.1 Get the list of animes

In [ ]:
def getUrls(start, stop): 

    urls = []
    for i in range(start, stop):
        url = 'https://myanimelist.net/topanime.php?limit='+str(i*50)
        r = requests.get(url)
        html_content = r.text
        soup = bs(html_content, 'lxml')
        links = soup.find_all('h3') 

        for anime in links[:-3]:
            if anime.find('a'):
                urls.append(anime.find('a')['href'])
    return urls

In [ ]:
f = open("topAnime.txt", 'w', encoding="utf8")
f.write('\n'.join(getUrls(0,400)))
f.close

## 1.2 Crawl animes

In [ ]:
def saveHtml(page):
#saving the HTML file of page 'page' in the corresponding folder

    subfolder = "downloaded_Html/page_{}".format(page)
    os.makedirs(subfolder)

    f = open("topAnime.txt", 'r', encoding="utf8")
    lines = f.readlines()[(page-1)*50:(page)*50]
    lines = [line.rstrip() for line in lines]
    f.close

    i = 1+50*(page-1)
    for link in lines:
        html = requests.get(link)
        
        if html.status_code != 200:
            while(html.status_code != 200): 
                check = requests.get(link)
        
        file_name = '{}/{}.html'.format(subfolder, i)
        g = open(file_name, 'w', encoding="utf8")
        g.write(html.text)
        g.close
        i += 1

In [ ]:
for i in range(379,400):
    saveHtml(i)

## 1.3 Parse downloaded pages

In [ ]:
def getTitle(anime):
    return anime.strong.contents[0]

def getType(anime):
    return anime.find(text = 'Type:').find_next('a').contents[0]

def getNumEpis(anime):
    if anime.find(text = 'Episodes:').next_element.strip() != "Unknown":
        return int(anime.find(text = 'Episodes:').next_element.strip())
    else :
        return []
    
def getStart(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    
    if date != "Not available":
        if len(date.split(" to ")[0]) > 8:
            return dt.strptime(date.split(" to ")[0], '%b %d, %Y' )
        elif len(date.split(" to ")[0]) == 4:
            return dt.strptime(date.split(" to ")[0], '%Y' )
        elif len(date.split(" to ")[0]) == 8:
            return dt.strptime(date.split(" to ")[0], '%b %Y' )
        else:
            return pd.to_datetime(np.NaN, errors='coerce')
    else :
        return []
    
def getEnd(anime):
    date = anime.find(text = 'Aired:').next_element.strip()
    
    if date != "Not available":
        if len(date)>12 and len(date.split(" to ")[1]) > 8 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%b %d, %Y' )
        elif len(date)>12 and len(date.split(" to ")[1]) == 4 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%Y' )
        elif len(date)>12 and len(date.split(" to ")[1]) == 8 and date.split(" to ")[1] != "?":
            return dt.strptime(date.split(" to ")[1], '%b %Y' )
        else:
            return pd.to_datetime(np.NaN, errors='coerce')
    else :
        return []        
    
def getNumMemb(anime):
    animeNumMembers = anime.find(text = 'Members:').next_element
    return int(animeNumMembers.replace('n', '').replace(',', '').strip())

def getScore(anime):
    if anime.find(text = 'Score:').find_next('span').contents[0] != 'N/A':
        animeScore = anime.find(text = 'Score:').find_next('span').contents
        return float(animeScore[0])
    else :
        return []         
    
def getUsers(anime):
    animeUsers = anime.find(text = 'Score:').find_next('span').find_next('span').contents
    if animeUsers[0] != 'Ranked:':
        return int(animeUsers[0])
    else :
        return []      

def getRank(anime):
    animeRank = anime.find(text = 'Ranked:').next_element
    if animeRank.replace('\n', '').strip() != 'N/A':
        return int(animeRank.replace('\n', '').replace('#', '').strip())
    else :
        return [] 
    
def getPopularity(anime):
    animePopularity = anime.find(text='Popularity:').next_element
    return int(animePopularity.replace("\n","").replace('#', '').strip())

def getDescription(anime):
    animeDescription = anime.find(text = 'Synopsis').find_next('p').text
    return animeDescription.replace("\n","")

def getRelated(anime):
    table = anime.find(text = 'Related Anime')
    animeRelated = []

    if(table != None):    
        table = table.find_next('table')
        table = table.find_all('a')

        for el in table:
            animeRelated.append(el.text)
                
    return animeRelated

def getCharact(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('table')

    animeChar = []

    for el in table:
        people = el.find_all('h3')
        for person in people:
            animeChar.append(person.text)
        
    return animeChar

def getVoices(anime):
    table = anime.find(text = 'Characters & Voice Actors').find_next('div')
    table = table.find_all('h3')

    animeVoices = []

    for el in table:
        people = el.find_next('table')
        for person in people:
            animeVoices.append(person.find('a').text)
        
    return animeVoices

def getStaff(anime):
    
    table = anime.find_all(text = 'Staff')[1].find_next("div", {"class": "detail-characters-list clearfix"})
    animeStaff = []
    if(table != None):    
        table = table.find_all("table")
        for el in table:
            x = el.find_all("td")[1]
            person = [x.find("a").text, x.find("small").text]
            animeStaff.append(person)
    
    return animeStaff

In [ ]:
pages = sorted(os.listdir('downloaded_Html')[1:], key = lambda page : int(page.split("_")[1]))

title = []
typ = []
numEpisode = []
start = []
end = []
numMembers = []
score = []
users = []
rank = []
popularity = []
synopsis = []
related = []
char = []
voices = []
staff = []

In [ ]:
for page in pages:
    htmls = os.listdir('downloaded_Html/{}'.format(page))
    for i in range(1,1+len(htmls)):
        f = open("downloaded_Html/{}/{}.html".format(page,50*(int(page.split("_")[1])-1)+i), 'r', encoding="utf8")
        anime = bs(f, 'lxml')
        title.append(getTitle(anime))
        typ.append(getType(anime))
        numEpisode.append(getNumEpis(anime))
        start.append(getStart(anime))
        end.append(getEnd(anime))
        numMembers.append(getNumMemb(anime))
        score.append(getScore(anime))
        users.append(getUsers(anime))
        rank.append(getRank(anime))
        popularity.append(getPopularity(anime))
        synopsis.append(getDescription(anime))
        related.append(getRelated(anime))
        char.append(getCharact(anime))
        voices.append(getVoices(anime))
        staff.append(getStaff(anime))

In [ ]:
col = ['animeTitle', 'animeType', 'animeNumEpisode','releaseDate', 'endDate', 'animeNumMembers', 'animeScore', 'animeUsers', 
       'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']

types = {'animeTitle' : 'object', 
         'animeType' : 'object', 
         'animeNumEpisode' : 'int64',
         'releaseDate' : 'datetime64', 
         'endDate' : 'datetime64', 
         'animeNumMembers' : 'int64', 
         'animeScore' : 'float64',
         'animeUsers' : 'int64', 
         'animeRank' : 'int64',
         'animePopularity' : 'int64',
         'animeDescription' : 'object',
         'animeRelated' : 'object',
         'animeCharacters' : 'object',
         'animeVoices' : 'object',
         'animeStaff' : 'object'}

df = pd.DataFrame(columns = col).astype(dtype = types) 

df.animeTitle = title 
df.animeType = typ 
df.animeNumEpisode = numEpisode 
df.releaseDate = start 
df.endDate = end 
df.animeNumMembers = numMembers 
df.animeScore = score 
df.animeUsers = users 
df.animeRank = rank 
df.animePopularity = popularity 
df.animeDescription = synopsis 
df.animeRelated = related 
df.animeCharacters = char 
df.animeVoices = voices 
df.animeStaff = staff 

In [ ]:
df = df.to_csv("data/anime.tsv", sep='\t', index=False)

# 2. Search engine

## 2.1 Conjunctive query

In [ ]:
synopsis = pd.read_csv("data/anime.tsv", sep='\t', usecols = "animeDescription")
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
tokenizer = RegexpTokenizer(r"\w+")
ps = PorterStemmer()

In [ ]:
def row_token(row):
    out = list()
    tokens = tokenizer.tokenize(row.lower())
    for word in tokens:
        if word not in stop_words:
            out.append(ps.stem(word))
    out = list(dict.fromkeys(out))
    return out

In [ ]:
total = []

for row in synopsis:
    for word in row_token(row):
        if word not in stop_words:
            total.append(ps.stem(word))

In [ ]:
voc = {}
id = 0
for tok in total:
    voc[tok] = id
    id+=1

with open('data/vocabulary.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['Word', 'term_id']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in voc:
        w.writerow({"Word": key, 'term_id': voc[key]})

In [ ]:
vocabulary = pd.read_csv("data/vocabulary.csv")
vocabulary.head()

In [ ]:
def setDict(d,l):
    for i in l:
        d[i] = []
    return d

def listToString(l):
    return ' '.join([str(elem) for elem in l])

In [ ]:
inverted_index = {}
inverted_index = setDict(inverted_index, vocabulary.term_id)

for i in range(len(synopsis)):
    s = listToString(row_token(synopsis[i]))
    for j in range(len(vocabulary.Word)):
        if s.find(str(vocabulary.Word[j])) != -1:
            inverted_index[vocabulary.term_id[j]].append(i)

In [ ]:
with open('data/inverted_index.csv', 'w', newline = '',encoding = 'utf8') as f:
    fieldnames = ['term_id', 'document']
    w = csv.DictWriter(f, fieldnames = fieldnames,)
    w.writeheader()
    for key in inverted_index:
        w.writerow({"term_id": key, 'document': inverted_index[key]})

In [ ]:
def searchWord(i):
    vocabulary = pd.read_csv("data/vocabulary.csv")
    return i in list(vocabulary.Word)

In [ ]:
def searchEngine(query):
    vocabulary = pd.read_csv("data/vocabulary.csv")
    index = pd.read_csv("data/inverted_index.csv")
    stemQuery = row_token(query)
    
    if all([searchWord(i) for i in stemQuery]):
        ind = [np.where(vocabulary.Word == x)[0][0] for x in stemQuery]
        aux = [set(ast.literal_eval(index.document[vocabulary.term_id[i]])) for i in ind]
        return set.intersection(*aux)

In [ ]:
def documentFinder(query):
    anime = pd.read_csv("data/anime.tsv", sep='\t', usecols = ["animeTitle","animeDescription"])
    f = open("data/topAnime.txt", 'r', encoding="utf8")
    topAnimeUrls = f.readlines()
    col = ['animeTitle', "animeDescription", "Url"]
    df = pd.DataFrame(columns = col)
    ind = searchEngine(query)
        
    df.animeTitle = [anime.animeTitle[i] for i in ind]
    df.animeDescription = [anime.animeDescription[i] for i in ind]
    df.Url = [topAnimeUrls[i] for i in ind]
    
    f.close
    return df 

In [ ]:
query = "race sayan"
documentFinder(query)

## 2.2 Conjunctive query & Ranking score

In [ ]:
import numpy as np
import pandas as pd 
import requests
import time
import csv
import os
import nltk
import ast
from nltk.corpus import stopwords
from bs4 import BeautifulSoup as bs
from datetime import datetime as dt
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from os.path import expanduser

In [ ]:
def wordnet_pos_gen(lista):
    pos_tags = list(nltk.pos_tag(lista))
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    t = []
    for x in pos_tags:
        try:
            t.append((x[0], tag_dict[x[1][0]]))
        except Exception:
            t.append(
                (x[0], "n"))  # a get around to assign irrelevant tags to one of wordnet classes, noun in this case
    return t

# 3. Define a new score!

# 4. Understanding the anime's reviews

# 5. Algorithmic question

We are askedd by a personal trainer who has a back-to-back sequence of requests for appointments, to provide a schedule that maximizes the total length of the accepted appointments avoiding consecutive ones. 

Our first idea is to analyze with a tree all the possible combination of the possible choices. As we can see in the photo below, until we have an array of length 5, the possible choices are forced:

![image4](images/image4.png)

Now that we want to put the fifth element of the array, we can take into account the possibility of appending that directly to the root, that is the first element of the array, but we realize that it makes no sense because:
- time is always positive;
- the fifth element can be appended to the third element since they are not consecutive

so when we want to append the fifth element, since we want to get maximizes the amount of time of the appointment, we put it under the third one.

![image2](images/image2.png)

In conclusion we get a binary tree of the shape:

<div>
<img src = "images/image6.png" width = "500px">
</div>

So in order to solve the problem assigned by the personal trainer we will have to build this type of trees. In particular we note that there are only two possible distinct casese the tree rooted in the first element and the one rooted in the second.

In [ ]:
# we create all the possibile path from 1 to n and we save it in lists
def possiblePath(index):
    lists = [[index[0]]]
    n = 2
    
    while n < len(index):
        for l in lists:
            # we add the element n+1
            if(n - np.where(index == l[-1])[0][0] == 2):
                l.append(index[n])
            # we add the element n+2
            if len(l)>1 and (n - np.where(index == l[-2])[0][0] == 3):
                aux = l[:-1]
                aux.append(index[n]) 
                if aux not in lists:
                    lists.append(aux)
        n +=1
    return lists

In [ ]:
# from the path of index obtained in the previous function 
# we  trasform each index in the corrispondent request value
def pathToRequest(path, request):
    
    for l in path:
        for i in range(len(l)):
            ind = l[i]-1
            l[i] = request[ind]
            
    return path

In [ ]:
# seraching for the best path between the 1-rooted tree vs the 2-rooted tree
def optimalPath(path1,path2):
    
    sums1 = np.array([sum(l) for l in path1])
    sums2 = np.array([sum(l) for l in path2])
    
    if(max(sums1)>=max(sums2)):
        max_value = np.where(sums1 == max(sums1))[0][0]
        return path1[max_value]
    else:
        max_value = np.where(sums2 == max(sums2))[0][0]
        return path2[max_value]

In [ ]:
import numpy as np

request = [30,40,25,50,30,20]
index = np.array([*range(1,len(request)+1)])

path1 = possiblePath(index)
path1 = pathToRequest(path1,request)

path2 = possiblePath(index[1:])
path2 = pathToRequest(path2,request)

request = optimalPath(path1,path2)
print("The best way to accept the appointments in the given order is: ", request ,"\nwith a total duration of :" , sum(request))